<a href="https://colab.research.google.com/github/josephmailil1/Speciale---Stibo/blob/main/Agent%20Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

In [2]:
from google.colab import userdata
openai_api_key = userdata.get('OpenAI_Key') #use the name of your colab key

In [3]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
import os
os.chdir("/content/drive/My Drive/Speciale")

In [6]:
shap = pd.read_csv("shap_noincome100.csv")
data = pd.read_csv("basedata100.csv")

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0, openai_api_key=openai_api_key)

In [8]:
from langchain_experimental.tools import PythonAstREPLTool

In [9]:
tools = [PythonAstREPLTool(locals={"data": data, "shap": shap})]

In [13]:
llm_with_tools = llm.bind_tools(tools)

In [14]:
from langchain.output_parsers import JsonOutputToolsParser

In [15]:
parser = JsonOutputToolsParser(tool = tools)

In [29]:
shap.head(10)

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,Row_Number
0,-0.006508,-0.000894,0.102855,-0.018275,-0.005681,-0.033657,0.000099,0.000370,-0.000686,0.001056,...,-0.000001,0.0,0.0,0.0,0.0,0.0,-0.014199,0.0,0.0,0
1,0.015186,0.001135,-0.031469,-0.013235,-0.004023,-0.016537,0.002856,0.004583,-0.001100,0.000308,...,-0.000014,0.0,0.0,0.0,0.0,0.0,0.000288,0.0,0.0,1
2,0.025775,0.003364,-0.027138,-0.018926,-0.006049,0.020340,-0.000882,0.007498,-0.001038,-0.011255,...,-0.000001,0.0,0.0,0.0,0.0,0.0,0.000925,0.0,0.0,2
3,0.016869,-0.000648,-0.030525,-0.017139,-0.004205,-0.018899,0.000128,0.003301,-0.001072,-0.002189,...,-0.000006,0.0,0.0,0.0,0.0,0.0,0.000320,0.0,0.0,3
4,-0.044365,-0.000126,-0.017828,-0.012122,-0.002964,-0.006371,-0.000054,-0.001044,-0.000420,0.000050,...,-0.000002,0.0,0.0,0.0,0.0,0.0,-0.001889,0.0,0.0,4
5,-0.004021,-0.000389,-0.026188,-0.015695,-0.004341,-0.013622,-0.000216,0.006218,-0.000970,0.000315,...,-0.000006,0.0,0.0,0.0,0.0,0.0,0.000246,0.0,0.0,5
6,0.028570,-0.001459,-0.040220,-0.019101,-0.006694,0.017822,-0.000081,-0.002098,-0.001050,0.001972,...,-0.000001,0.0,0.0,0.0,0.0,0.0,0.000698,0.0,0.0,6
7,-0.024039,-0.004518,-0.059785,-0.017443,-0.005231,-0.011680,-0.000161,0.010739,-0.000871,0.001179,...,-0.000002,0.0,0.0,0.0,0.0,0.0,0.000739,0.0,0.0,7
8,-0.045670,0.000082,-0.024010,-0.012037,-0.002886,-0.011302,-0.000056,-0.001035,-0.000496,0.000045,...,-0.000002,0.0,0.0,0.0,0.0,0.0,0.000339,0.0,0.0,8
9,0.029039,0.000962,0.129079,-0.017752,-0.004665,0.024029,0.001686,-0.001744,-0.001464,0.000233,...,-0.000001,0.0,0.0,0.0,0.0,0.0,0.000198,0.0,0.0,9


In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""You have access to a pandas dataframe `data`. \
            Here is the output of `data.head().to_markdown()`:

              ```
            {data.head().to_markdown()}
            ```
            You are also given the pandas dateframe shap. Containing shap values for the test set obsevations.
            The shap obsevation shows which values are important for the final prediction
            ```
            {shap.head().to_markdown()}
            ```

            If asked about the importance/relevance of obsevations or feature use the shap dataframe to answer.
            A high absolute values of the shap dataset means that the value is relevant

            Use the data dataframe to answer question about what the input values to the obsevation is.

            Respond directly to the question once you have enough information to answer it.""",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [22]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [41]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps = False)

In [25]:
import warnings
warnings.filterwarnings('ignore')

In [43]:
test = agent_executor.invoke({"input": "What are the five higest Shap values for obsevation 10"})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\ndata = pd.read_csv('data.csv')\nshap = pd.read_csv('shap.csv')\nshap_values = shap.loc[10].drop('Row_Number')\nshap_values.nlargest(5)"}`


FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'
Invoking: `python_repl_ast` with `{'query': "shap_values = shap.loc[10].drop('Row_Number')\nshap_values.nlargest(5)"}`
responded: It seems there was an error in accessing the data. Let me correct this and retrieve the five highest SHAP values for observation 10.

education-num                   0.073776
occupation_Exec-managerial      0.059783
education_Bachelors             0.050896
marital-status_Never-married    0.027138
age                             0.019383
Name: 10, dtype: float64The five highest SHAP values for observation 10 are:

1. `education-num`: 0.073776
2. `occupation_Exec-managerial`: 0.059783
3. `education_Bachelors`: 0.050896
4. `marital-status_Never-married`

In [48]:
def call_agent(user_question):
    answer = agent_executor.invoke({"input": user_question})
    return answer['output']

In [49]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 57.0 MB/s

In [50]:
import gradio as gr

In [51]:
demo = gr.Interface(
    fn=call_agent,
    inputs=["text"],
    outputs=["text"]
)

In [52]:
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6ee99dde1c0ba7ac1b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
agent = create_pandas_dataframe_agent(llm, [shap, data], verbose=True)
agent.invoke("how many rows in the age column are different?")

In [36]:
import pandas as pd


# Choose the row you want to find the highest values for
row_index = 10  # For example, choose the first row

# Get the row as a Series
row = shap.iloc[row_index]

# Sort the row values in descending order and get the top 5
top_5_values = row.sort_values(ascending=False).head(5)

print("Top 5 highest values in the row:")
print(top_5_values)

Top 5 highest values in the row:
Row_Number                      10.000000
education-num                    0.073776
occupation_Exec-managerial       0.059783
education_Bachelors              0.050896
marital-status_Never-married     0.027138
Name: 10, dtype: float64
